# Ryhmä-190

## Python-paketit

In [1]:
# Jupyter-notebookissa voi asentaa paketit samaan tapaan kuin komentoriviltä.
# Tähän käytetään "magic commandeja", jotka alkavat %-merkillä.
# Näiden asentamisessa voi mennä muutama minuutti, mutta sen jälkeen niitä ei tarvitse asentaa uudestaan. 
#%pip install seaborn
#%pip install mlflow azureml azureml-core azureml-mlflow azure-identity

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.model_selection import KFold, cross_val_score
#from azureml.core import Workspace
#from azure.identity import InteractiveBrowserCredential
#import mlflow

## MLFlow-seuranta (ei tällä hetkellä käytössä)

Tarkistetaan ajoympäristö (Azure vai oma kone) ja autentikoidutaan sen mukaan.

In [3]:
def is_running_in_azure():
    return 'AZUREML_RUN_ID' in os.environ

def get_workspace():
    if is_running_in_azure():
        return Workspace.from_config()
    else:
        return Workspace(subscription_id='1c0e26b6-0fcb-4b6d-911c-2a0836275ea4',
                         resource_group='rg-AML',
                         workspace_name='aml-data_science_masters')
#ws = get_workspace()

In [79]:
#mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
#experiment_name = 'group-190-tracking'
#mlflow.set_experiment(experiment_name)
#mlflow.autolog()

## Tietoaineistojen lataaminen

In [80]:
df_train = pd.read_csv('../data/train.csv', encoding='utf-8', header=0)
df_test = pd.read_csv('../data/test.csv', encoding='utf-8', header=0)

## Esiprosessointi

In [81]:
df_train['parentspecies'] = df_train['parentspecies'].astype('category')
df_train['parentspecies'] = df_train['parentspecies'].cat.codes
df_test['parentspecies'] = df_test['parentspecies'].astype('category')
df_test['parentspecies'] = df_test['parentspecies'].cat.codes

## Mallien kouluttaminen

In [82]:
X_train, y_train = df_train.drop(columns=['log_pSat_Pa', 'ID']), df_train['log_pSat_Pa']
X_test = df_test.drop(columns=['ID'])

# Feature Selection
Tässä voi kokeilla, miten vaikuttaa, jos jättää joitain featureita pois.  
Strongest correlation between the target:
| Feature | correlation with target |
|---------|:-----------------------:|
| NumHBondDonors|           0.689196|
|NumOfConf|                0.513653
|hydroperoxide |           0.314053
|hydroxyl (alkyl)   |      0.310452
|NumOfAtoms          |     0.307337
|carboxylic acid      |    0.304259
|NumOfC                |   0.262769
|carbonylperoxynitrate  |  0.223739
|MW                    |   0.199574
|NumOfN                |   0.183152

---------------------
Strongest correlation pairs: 
| Feature | Feature | Correlation |
|---------|---------|-------------|
|NumOfO       |          MW          |      0.880358 |
|NumOfC       |          NumOfAtoms   |     0.838402
|NumOfN        |         MW           |     0.772575
|NumOfAtoms    |         MW           |     0.707009
|nitrate       |         NumOfN       |     0.687224
|NumOfN        |         NumOfO       |     0.656750
|hydroxyl (alkyl)|       NumHBondDonors |   0.632023
|carbonylperoxynitrate|  NumOfO         |   0.510409
|nitrate              |  NumOfAtoms     |   0.492108
|NumOfN                | NumOfAtoms     |   0.491902

In [ ]:
X_train = X_train.drop(columns=['MW', 'aromatic hydroxyl', 'C=C-C=O in non-aromatic ring', 'nitro', 'peroxide', 'C=C (non-aromatic)', 'NumOfC'])
X_test = X_test.drop(columns=['MW', 'aromatic hydroxyl', 'C=C-C=O in non-aromatic ring', 'nitro', 'peroxide', 'C=C (non-aromatic)', 'NumOfC'])

Erilaisia malleja alla. Kaikki mallit käyttävät samaa nimeä `model`, eli vain viimeisenä koulutettu menee testeihin ja tallentuu.

### 1. Dummy

In [83]:
model = DummyRegressor(strategy='mean')
model.fit(X_train, y_train)

df_test['log_pSat_Pa'] = model.predict(X_test)

2024/11/21 16:18:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9aaabb6b-85b1-4567-ad8a-c085e5cdd601', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/11/21 16:18:35 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\mikkokallio\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Han

🏃 View run sad_kitchen_pc72hd6y at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/1c0e26b6-0fcb-4b6d-911c-2a0836275ea4/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/aml-data_science_masters/#/experiments/e59c04c4-74a6-4c62-86e6-370ea69518dd/runs/9aaabb6b-85b1-4567-ad8a-c085e5cdd601
🧪 View experiment at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/1c0e26b6-0fcb-4b6d-911c-2a0836275ea4/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/aml-data_science_masters/#/experiments/e59c04c4-74a6-4c62-86e6-370ea69518dd


2024/11/21 16:18:54 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 9aaabb6b-85b1-4567-ad8a-c085e5cdd601. Failed operations: [RestException(\'BAD_REQUEST: Response: {\\\'Error\\\': {\\\'Code\\\': \\\'UserError\\\', \\\'Severity\\\': None, \\\'Message\\\': "Could not log all inputs as datasets:\\\\nCREATE can\\\'t modify existing entity, please use UPDATE.", \\\'MessageFormat\\\': None, \\\'MessageParameters\\\': None, \\\'ReferenceCode\\\': None, \\\'DetailsUri\\\': None, \\\'Target\\\': None, \\\'Details\\\': [], \\\'InnerError\\\': None, \\\'DebugInfo\\\': None, \\\'AdditionalInfo\\\': None}, \\\'Correlation\\\': {\\\'operation\\\': \\\'9a427ceb853959bb0e098d3b0db531a1\\\', \\\'request\\\': \\\'62d8ac1009c697c7\\\'}, \\\'Environment\\\': \\\'swedencentral\\\', \\\'Location\\\': \

### 2. Yksinkertainen regressio

In [84]:
model = LinearRegression()
model.fit(X_train, y_train)

df_test['log_pSat_Pa'] = model.predict(X_test)

2024/11/21 16:18:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '747030c3-d398-4ed6-a0c5-2eb4e6a5a6f6', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/11/21 16:18:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\mikkokallio\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Han

🏃 View run clever_stone_g5hxwq6c at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/1c0e26b6-0fcb-4b6d-911c-2a0836275ea4/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/aml-data_science_masters/#/experiments/e59c04c4-74a6-4c62-86e6-370ea69518dd/runs/747030c3-d398-4ed6-a0c5-2eb4e6a5a6f6
🧪 View experiment at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/1c0e26b6-0fcb-4b6d-911c-2a0836275ea4/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/aml-data_science_masters/#/experiments/e59c04c4-74a6-4c62-86e6-370ea69518dd


2024/11/21 16:19:07 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 747030c3-d398-4ed6-a0c5-2eb4e6a5a6f6. Failed operations: [RestException(\'BAD_REQUEST: Response: {\\\'Error\\\': {\\\'Code\\\': \\\'UserError\\\', \\\'Severity\\\': None, \\\'Message\\\': "Could not log all inputs as datasets:\\\\nCREATE can\\\'t modify existing entity, please use UPDATE.", \\\'MessageFormat\\\': None, \\\'MessageParameters\\\': None, \\\'ReferenceCode\\\': None, \\\'DetailsUri\\\': None, \\\'Target\\\': None, \\\'Details\\\': [], \\\'InnerError\\\': None, \\\'DebugInfo\\\': None, \\\'AdditionalInfo\\\': None}, \\\'Correlation\\\': {\\\'operation\\\': \\\'07a3841385123d18238ed69005e6a755\\\', \\\'request\\\': \\\'a36fb0fd9f885023\\\'}, \\\'Environment\\\': \\\'swedencentral\\\', \\\'Location\\\': \

### 3. Random Forest

In [85]:
model = RandomForestRegressor(random_state=190)
model.fit(X_train, y_train)

df_test['log_pSat_Pa'] = model.predict(X_test)

2024/11/21 16:19:07 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '90a1ec9c-88d9-4c82-9f00-4ff8be0ddd00', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/11/21 16:19:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\mikkokallio\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Han

🏃 View run jovial_crowd_9tccpyfm at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/1c0e26b6-0fcb-4b6d-911c-2a0836275ea4/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/aml-data_science_masters/#/experiments/e59c04c4-74a6-4c62-86e6-370ea69518dd/runs/90a1ec9c-88d9-4c82-9f00-4ff8be0ddd00
🧪 View experiment at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/1c0e26b6-0fcb-4b6d-911c-2a0836275ea4/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/aml-data_science_masters/#/experiments/e59c04c4-74a6-4c62-86e6-370ea69518dd


2024/11/21 16:20:05 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 90a1ec9c-88d9-4c82-9f00-4ff8be0ddd00. Failed operations: [RestException(\'BAD_REQUEST: Response: {\\\'Error\\\': {\\\'Code\\\': \\\'UserError\\\', \\\'Severity\\\': None, \\\'Message\\\': "Could not log all inputs as datasets:\\\\nCREATE can\\\'t modify existing entity, please use UPDATE.", \\\'MessageFormat\\\': None, \\\'MessageParameters\\\': None, \\\'ReferenceCode\\\': None, \\\'DetailsUri\\\': None, \\\'Target\\\': None, \\\'Details\\\': [], \\\'InnerError\\\': None, \\\'DebugInfo\\\': None, \\\'AdditionalInfo\\\': None}, \\\'Correlation\\\': {\\\'operation\\\': \\\'c185a95272dbd1df145f9cd6338a5260\\\', \\\'request\\\': \\\'e23a1c7374a2966f\\\'}, \\\'Environment\\\': \\\'swedencentral\\\', \\\'Location\\\': \

### 4. Gradient Boosting Regressor

In [86]:
model = GradientBoostingRegressor(random_state=190)
model.fit(X_train, y_train)

df_test['log_pSat_Pa'] = model.predict(X_test)

2024/11/21 16:20:05 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'fa8ad584-0749-4e84-bcfa-dc26f2381b59', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/11/21 16:20:05 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\mikkokallio\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Han

🏃 View run blue_garlic_qdqyhgjn at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/1c0e26b6-0fcb-4b6d-911c-2a0836275ea4/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/aml-data_science_masters/#/experiments/e59c04c4-74a6-4c62-86e6-370ea69518dd/runs/fa8ad584-0749-4e84-bcfa-dc26f2381b59
🧪 View experiment at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/1c0e26b6-0fcb-4b6d-911c-2a0836275ea4/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/aml-data_science_masters/#/experiments/e59c04c4-74a6-4c62-86e6-370ea69518dd


2024/11/21 16:20:19 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID fa8ad584-0749-4e84-bcfa-dc26f2381b59. Failed operations: [RestException(\'BAD_REQUEST: Response: {\\\'Error\\\': {\\\'Code\\\': \\\'UserError\\\', \\\'Severity\\\': None, \\\'Message\\\': "Could not log all inputs as datasets:\\\\nCREATE can\\\'t modify existing entity, please use UPDATE.", \\\'MessageFormat\\\': None, \\\'MessageParameters\\\': None, \\\'ReferenceCode\\\': None, \\\'DetailsUri\\\': None, \\\'Target\\\': None, \\\'Details\\\': [], \\\'InnerError\\\': None, \\\'DebugInfo\\\': None, \\\'AdditionalInfo\\\': None}, \\\'Correlation\\\': {\\\'operation\\\': \\\'5d821410f42f0aa3be85f9836396eb24\\\', \\\'request\\\': \\\'d0f04a6f6e790333\\\'}, \\\'Environment\\\': \\\'swedencentral\\\', \\\'Location\\\': \

## Mallien arviointi

In [87]:
### Train loss
y_train_pred = model.predict(X_train)
train_loss = mean_squared_error(y_train, y_train_pred)

### 5-fold cross-validation
results = []
kf = KFold(n_splits=5, shuffle=True, random_state=190)
mse_scorer = make_scorer(mean_squared_error)
cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=mse_scorer)
cv_loss_mean = cv_scores.mean()

r2_train = r2_score(y_train, y_train_pred)
r2_cv = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2').mean()

print('Train loss:', train_loss)
print('CV loss mean:', cv_loss_mean)
print('Train R^2:', r2_train)
print('CV R^2:', r2_cv)

Train loss: 2.6116312914299806
CV loss mean: 2.6794213727758844
Train R^2: 0.7317339554862234
CV R^2: 0.7246891200107113


## Ennustuksen tallentaminen

In [88]:
df_test[['ID', 'log_pSat_Pa']].to_csv('../submission/submission.csv', index=False)